In [12]:
__author__ = 'SherlockLiao'

import os

import torch
import torchvision
from torch import nn, cuda
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
#from torchvision.datasets import MNIST
from torchvision.utils import save_image

#if not os.path.exists('./mlp_img'):
#    os.mkdir('./mlp_img')


def to_img(x):
    x = 0.5 * (x + 1)
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 1, 28, 28)
    return x


num_epochs = 100
batch_size = 128
learning_rate = 1e-3

img_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

device = 'cuda' if cuda.is_available() else 'cpu'
print(f'Training MNIST Model on {device}\n{"=" * 44}')
    # MNIST Dataset
train_dataset = datasets.MNIST(root='./mnist_data/',
                               train=True,
                               transform=img_transform,
                               download=True)

#test_dataset = datasets.MNIST(root='./mnist_data/',
#                              train=False,
#                              transform=img_transforms)
    # Data Loader (Input Pipeline)
dataloader = DataLoader(dataset=train_dataset,
                                       batch_size=batch_size,
                                       shuffle=True)

#test_loader = data.DataLoader(dataset=test_dataset,
#                                      batch_size=batch_size,
#                                      shuffle=False)
#dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


Training MNIST Model on cuda


In [13]:
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(28 * 28, 128),
            nn.ReLU(True),
            nn.Linear(128, 64),
            nn.ReLU(True), nn.Linear(64, 12), nn.ReLU(True), nn.Linear(12, 3))
        self.decoder = nn.Sequential(
            nn.Linear(3, 12),
            nn.ReLU(True),
            nn.Linear(12, 64),
            nn.ReLU(True),
            nn.Linear(64, 128),
            nn.ReLU(True), nn.Linear(128, 28 * 28), nn.Tanh())

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


model = autoencoder().cuda()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(
    model.parameters(), lr=learning_rate, weight_decay=1e-5)

for epoch in range(num_epochs):
    for data in dataloader:
        img, _ = data
        img = img.view(img.size(0), -1)
        img = Variable(img).cuda()
        # ===================forward=====================
        output = model(img)
        loss = criterion(output, img)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'
          .format(epoch + 1, num_epochs, loss.data[0]))
    if epoch % 10 == 0:
        pic = to_img(output.cpu().data)
        save_image(pic, './mlp_img/image_{}.png'.format(epoch))

torch.save(model.state_dict(), './sim_autoencoder.pth')

RuntimeError: output with shape [1, 28, 28] doesn't match the broadcast shape [3, 28, 28]